In [2]:
import zipfile 
import os
import glob
import re
import shutil
import csv
import gzip
from tqdm.notebook import tqdm

# TC 

- Stock

path : `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\stock`

On classe les csv selon leur balise:

- PM
- PP
- REP
- ETS
- OBS
- ACTES
- CA

Faire un log pour etre sur que chaque csv a les mêmes colonnes

In [ ]:
def moveCSVStock(pathcsv,pathsave):
    """
    """
    log_columns = []
    for file in glob.glob("{}\*.csv".format(pathcsv)):
        for i in ["PM", "PP", "rep", "ets", "actes", "comptes_annuels", "obs"]:
            #
            matches = re.search(i, file)
            if matches:
                dic_files_col = {}
                filename = os.path.basename(file)
                new_des = "{}\{}\{}".format(pathsave, i.upper(), filename)
                with open(filename, "r", encoding="utf-8") as f:
                    reader = csv.reader(f)

                    col = next(reader)
                    dic_files_col["filename"] = filename
                    dic_files_col["origin"] = i
                    dic_files_col["columns"] = col
                    row_count = sum(1 for line in f)
                    dic_files_col["rows_count"] = row_count
                log_columns.append(dic_files_col)
                ### save as gzip
                shutil.move(file, new_des)

    return log_columns

## Stock

Il y a deux types de stocks: 

- Le stock initial constitué à la date du 4 mai 2017
- Le stock partiel
    * Des stocks partiels constitués des dossiers complets relivrés à la demande de l’INPI après détection d’anomalies ?
    *  Les fichiers des données contenues dans les nouvelles inscriptions (immatriculations, modifications et radiations) du Registre national du commerce et des sociétés ainsi que les informations relatives aux dépôts des actes et comptes annuels, telles que transmises par les greffes à compter du 5 mai 2017 (données de flux).

### Stock initial

In [ ]:
path = r"G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\stock\2017\05\04"
list_zip = []
for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
    for name in files:
        if name.endswith((".zip")):
            path_xml = "{}\{}".format(
                    root,
                    name)
            list_zip.append(path_xml)

In [ ]:
len(list_zip)

In [ ]:
pathsave = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Stock\Stock_initial"
pathcsv = os.getcwd()
list_columns_csv = []
for file in tqdm(list_zip):
    zipfilePath = (file)
    zip = zipfile.ZipFile(zipfilePath)
    zip.extractall(".")
    zip.close()
    col = moveCSVStock(pathcsv, pathsave)
    list_columns_csv.append(col)

In [ ]:
len(list_columns_csv)

In [ ]:
with open('conformiter_stock_initial.json', 'w') as outfile:
    json.dump(list_columns_csv, outfile)

### Stock partiel

Les données de 2019 ne sont pas disponibles entièrement

In [ ]:
for year in [2018, 2019]:
    path = r"G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\stock\{}".format(year)
    list_zip = []
    for root, dirs, files in os.walk(path):
        # Test-> on veut pas avant mai 2018
        for name in files:
            if name.endswith((".zip")):
                path_xml = "{}\{}".format(
                    root,
                    name)
                list_zip.append(path_xml)

In [ ]:
len(list_zip)

In [ ]:
pathsave = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Stock\Stock_partiel"
pathcsv = os.getcwd()
list_columns_csv = []
for file in tqdm(list_zip):
    zipfilePath = (file)
    zip = zipfile.ZipFile(zipfilePath)
    zip.extractall(".")
    zip.close()
    col = moveCSVStock(pathcsv, pathsave)
    list_columns_csv.append(col)

## Flux

Données de 2018 et 2019 incomplètent.

- Les données de 2018 ne proviennent pas de l'INPI mais de http://data.cquest.org/inpi_rncs/imr/
- Données 2019 manquantes après mai

Pour la conformité, ouvrir `Conformite.ipynb`

In [ ]:
path = r"G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\05"
list_zip = []
for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
    for name in files:
        if name.endswith((".csv")):
            path_xml = "{}\{}".format(
                    root,
                    name)
            list_zip.append(path_xml)

In [ ]:
len(list_zip)

In [ ]:
#with open('listtomove.json', 'w') as outfile:
#    json.dump(list_zip, outfile)

In [4]:
#from shutil import copyfile
def moveCSVFlux(file,pathsave):
    """
    """
    log_columns = []
    #for file in glob.glob("{}\*.csv".format(pathcsv)):
    for i in ["PM", "PP", "rep", "ets", "actes", "comptes_annuels", "obs"]:
        filename = os.path.basename(file)
        matches = re.search(i, filename)
        if matches:
            ### Check if EVT, if EVT, move to EVT folder
            matches_evt = re.search("EVT", filename)
            if matches_evt:
                new_des = "{}\{}\EVT\{}".format(pathsave, i.upper(), filename)
            else:
                new_des = "{}\{}\{}\{}".format(pathsave, i.upper(),'NEW', filename)
                
            dic_files_col = {}
            
            #print(new_des)
            with open(file, "r", encoding="utf-8") as f:
                reader = csv.reader(f)

                col = next(reader)
                dic_files_col["filename"] = filename
                dic_files_col["origin"] = i
                dic_files_col["columns"] = col
                row_count = sum(1 for line in f)
                dic_files_col["rows_count"] = row_count
            log_columns.append(dic_files_col)
                ### save as gzip
            shutil.copyfile(file, new_des)

    return log_columns

In [ ]:
list_zip[:1]

In [ ]:
pathsave = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017"
pathcsv = os.getcwd()
list_columns_csv = []
list_failed = []
for file in tqdm(list_zip):
    try:
        col = moveCSVFlux(file, pathsave)
    except:
        list_failed.append(file)
    list_columns_csv.append(col)

Problème avec les deux fichiers suivants:

- ['G:\\Projets\\PP2019\\2019_PROGRAMME BATICA\\DATA\\INPI\\sources\\IMR_Donnees_Saisies\\tc\\Flux\\2017\\05\\24\\5601\\5\\5601_5_20170512_213441_11_obs.csv',
- 'G:\\Projets\\PP2019\\2019_PROGRAMME BATICA\\DATA\\INPI\\sources\\IMR_Donnees_Saisies\\tc\\Flux\\2017\\05\\24\\8401\\5\\8401_5_20170512_212823_11_obs.csv'

In [ ]:
list_failed

In [ ]:
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\05\24\5601\5
    
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\05\24\8401\5

### Batch

On doit faire de 05 à 12

In [ ]:
%%time
list_failed = []
for mois in tqdm(["05","06", "07", "08", "09", "10", "11", "12"]):
    list_zip = []
    path = r"G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\{}".format(mois)
    pathsave = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\{}".format(mois)
    for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
        for name in files:
            if name.endswith((".csv")):
                path_xml = "{}\{}".format(
                    root,
                    name)
                list_zip.append(path_xml)
                
    pathcsv = os.getcwd()
    list_columns_csv = []
    
    for file in tqdm(list_zip):
        try:
            col = moveCSVFlux(file, pathsave)
        except:
            list_failed.append(file)
        list_columns_csv.append(col)

In [21]:
len(list_failed)

Wall time: 0 ns


286870

In [17]:
list_zip[0]

'G:\\Projets\\PP2019\\2019_PROGRAMME BATICA\\DATA\\INPI\\sources\\IMR_Donnees_Saisies\\tc\\Flux\\2017\\12\\01\\0101\\142\\0101_142_20171201_095449_10_ets_supprime_EVT.csv'

In [16]:
pathsave

'C:\\Users\\PERNETTH\\Documents\\Projects\\GitLab\\CreationDataBase\\INPI\\TC_1\\Flux\\2017\\12'

In [19]:
moveCSVFlux(list_zip[0], pathsave)

[{'filename': '0101_142_20171201_095449_10_ets_supprime_EVT.csv',
  'origin': 'ets',
  'columns': ['\ufeffCode Greffe;Nom_Greffe;Numero_Gestion;Siren;Type;Siège_PM;RCS_Registre;Adresse_Ligne1;Adresse_Ligne2;Adresse_Ligne3;Code_Postal;Ville;Code_Commune;Pays;Domiciliataire_Nom;Domiciliataire_Siren;Domiciliataire_Greffe;Domiciliataire_Complément;Siege_Domicile_Représentant;Nom_Commercial;Enseigne;Activité_Ambulante;Activité_Saisonnière;Activité_Non_Sédentaire;Date_Début_Activité;Activité;Origine_Fonds;Origine_Fonds_Info;Type_Exploitation;ID_Etablissement;Date_Greffe;Libelle_Evt'],
  'rows_count': 1}]

In [22]:
%%time
for mois in tqdm(["05","06", "07", "08", "09", "10", "11", "12"]):
    #list_zip = []
    path = r"G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\{}".format(mois)
    pathsave = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\{}".format(mois)
    print(path)
    print(pathsave)

G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\05
C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\05
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\06
C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\06
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\07
C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\07
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\08
C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\08
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\09
C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\09
G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux\2017\10
C:\Users\PERNETT

In [ ]:
with open('conformiteflux.json', 'w') as outfile:
    json.dump(list_columns_csv, outfile)

## Remove all CSV

In [9]:
path = os.getcwd()
pathsave = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux"
for root, dirs, files in os.walk(pathsave):
    # Test-> on veut pas avant mai 2018
    for name in files:
        if name.endswith((".csv")):
            toremove = "{}\{}".format(root, name)
            os.remove(toremove)